## Noteable Plugin

```bash
pip install chatlab[noteable]
```

ChatLab can optionally be installed with a Noteable `NotebookClient`. Using this, you can re-create the Noteable Plugin experience in your own Jupyter Notebook environment. The `NotebookClient` allows you to pass only the functionsw you want, allowing you to tailor how the LLM responds. There's also a significant speed improvement over the Noteable Plugin because:

- The `NotebookClient` maintains a copy of the realtime notebook, allowing for faster cell creation and retrieval
- The one notebook per conversation model allows for faster LLM operations since it doesn't have to "type out" file IDs and project IDs continuously

You can even use this to create your own isolated version of the code interpreter. It's up to you to mix and match the functions you want to expose to the LLM.


In [14]:
from chatlab import models, Chat, FunctionRegistry, system
from chatlab.builtins.noteable import NotebookClient

In order to get the closest experience to the plugin, we'll grab the "plugin prompt" that comes in the Noteable manifest for the ChatGPT Plugin. We can then provide this in a `system` prompt for the model.


In [15]:
import requests

plugin_prompt = requests.get("https://chat.noteable.io/.well-known/ai-plugin.json").json()['description_for_model']

print(plugin_prompt[:90].strip() + "...")

On https://app.noteable.io, create and run Jupyter notebooks with code, markdown, and SQL...


In [16]:
import os

registry = FunctionRegistry()


async def create_notebook(file_name: str):
    """Create a notebook to use in this conversation"""
    nc = await NotebookClient.create(file_name=file_name, token=os.environ.get("NOTEABLE_TOKEN"))

    # Register all the regular notebook operations
    registry.register_functions(nc.chat_functions)
    # Let the model do `python` (which creates and runs a cell)
    registry.python_hallucination_function = nc.python

    return f"Notebook created at {nc.notebook_url}"


registry.register(create_notebook)

{'name': 'create_notebook',
 'description': 'Create a notebook to use in this conversation',
 'parameters': {'type': 'object',
  'properties': {'file_name': {'type': 'string'}},
  'required': ['file_name']}}

In [17]:
chat = Chat(system(plugin_prompt), model=models.GPT_3_5_TURBO_16K_0613, function_registry=registry)

In [18]:
await chat(
    "Let's make a notebook to analyze the Berkeley 311 calls from https://data.cityofberkeley.info/resource/bscu-qpbu.json. Please run code, perform analysis, and do the full EDA lifecycle for me. I recommend running the cells yourself as I want you to make sure to work based on the actual data format. Show me your best data scientist."
)

Received un-modeled RTU message msg.channel='kernels/notebook-kernel-e59108a5344f45a3af90' msg.event='variable_explorer_update_request'


Received un-modeled RTU message msg.channel='kernels/notebook-kernel-e59108a5344f45a3af90' msg.event='variable_explorer_update_request'


In the notebook [Berkeley_311_Analysis.ipynb](https://app.noteable.io/f/e59108a5-344f-45a3-af90-c40d964db9e9/Berkeley_311_Analysis.ipynb), I have performed the following analysis on the Berkeley 311 calls data:

1. Imported necessary libraries.
2. Fetched the Berkeley 311 calls data from the provided URL.
3. Created a pandas DataFrame from the data.
4. Checked the data types of the columns.
5. Counted the number of missing values in each column.
6. Dropped unnecessary columns from the DataFrame.
7. Converted date columns to datetime format.
8. Checked the data types of columns again.
9. Counted the number of missing values in each column again.
10. Checked the unique values in categorical columns.
11. Plotted the distribution of case status, request categories, and object types.
12. Grouped the data by case status and calculated the average time to close the cases.
13. Plotted the average time to close the cases by case status.

You can view the complete analysis with code and visualizations in the notebook. Let me know if there is anything specific you would like me to further analyze or if you have any other questions.